<a href="https://colab.research.google.com/github/yujinhor/ML_emergency-stop-prediction-pybullet/blob/main/accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from google.colab import drive

# 1. 구글 드라이브 연결
if not os.path.exists('/content/drive'):
    print("📂 구글 드라이브 연결 중...")
    drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/Robot_Analysis'

# 파일 경로
PRED_PATH = os.path.join(BASE_DIR, 'danger_max_duration_summary.csv')  # AI 예측 파일
LABEL_PATH = os.path.join(BASE_DIR, 'val_summary.csv')                 # 실제 정답 파일
OUTPUT_PATH = os.path.join(BASE_DIR, 'simple_match_result.csv')        # 결과 저장

def main():
    print("📖 데이터 읽는 중...")

    # 데이터 로드
    df_pred = pd.read_csv(PRED_PATH)
    df_label = pd.read_csv(LABEL_PATH)

    # 2. 딱 필요한 정보만 뽑아서 병합 (Episode_ID 기준)
    # AI 파일에는 'Episode_ID', 정답 파일에는 'episode'로 되어 있을 수 있음

    # 정답 파일에서 에피소드 번호와 결과(result_is_failure)만 가져옴
    # (result_is_failure: 1=고장, 0=성공)
    label_subset = df_label[['episode', 'result_is_failure']].copy()
    label_subset.rename(columns={'episode': 'Episode_ID'}, inplace=True)

    # 병합 (Left Join: AI가 위험하다고 한 목록 기준)
    merged_df = pd.merge(df_pred[['Episode_ID', 'Avg_Prob']], label_subset, on='Episode_ID', how='left')

    # 3. 결과 판정 (O/X 채점)
    # AI는 이미 위험하다고 예측했으므로, 실제가 1이면 정답, 0이면 틀린 것임
    merged_df['Check'] = merged_df['result_is_failure'].apply(
        lambda x: 'O (정답)' if x == 1 else 'X (오진)'
    )

    # 보기 좋게 컬럼 이름 변경
    merged_df.columns = ['에피소드_번호', 'AI_예측확률', '실제_결과(1=고장)', '판정']

    # 4. 저장 및 출력
    merged_df.to_csv(OUTPUT_PATH, index=False, encoding='utf-8-sig')

    print(f"\n✅ 매칭 완료! 저장된 파일: {OUTPUT_PATH}")
    print("\n[결과 미리보기]")
    print(merged_df.head(10)) # 상위 10개 출력

    # 5. 간단 요약
    correct = len(merged_df[merged_df['실제_결과(1=고장)'] == 1])
    total = len(merged_df)
    print(f"\n📊 요약: AI가 위험하다고 한 {total}개 중 {correct}개가 실제로 고장남.")
    print(f"   -> 적중률: {(correct/total)*100:.1f}%")

if __name__ == "__main__":
    main()

📖 데이터 읽는 중...

✅ 매칭 완료! 저장된 파일: /content/drive/MyDrive/Robot_Analysis/simple_match_result.csv

[결과 미리보기]
   에피소드_번호   AI_예측확률  실제_결과(1=고장)      판정
0        5  0.988862            1  O (정답)
1        6  0.989561            1  O (정답)
2        7  0.988293            1  O (정답)
3        8  0.912907            1  O (정답)
4       11  0.989370            1  O (정답)
5       12  0.808132            1  O (정답)
6       21  0.962442            1  O (정답)
7       24  0.989074            1  O (정답)
8       29  0.988117            1  O (정답)
9       30  0.964231            1  O (정답)

📊 요약: AI가 위험하다고 한 657개 중 575개가 실제로 고장남.
   -> 적중률: 87.5%
